In [ ]:
import yfinance as yf
import numpy as np
import random
import pandas as pd


In [2]:
# === 下載股票資料 ===

def download_stock_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    return data

# 指定股票清單（以科技類股為例，可自行替換）
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META", "ADBE", "ORCL", "INTC"]
start_date = "2020-01-01"
end_date = "2023-01-01"

# 下載股票價格資料
stock_data = download_stock_data(tickers, start_date, end_date)
stock_data = stock_data.dropna()  # 去除缺失值

[*********************100%***********************]  10 of 10 completed


In [3]:
# === 計算回報率和風險 ===

# 計算每日回報率
daily_returns = stock_data.pct_change().dropna()
mean_returns = daily_returns.mean()
std_dev = daily_returns.std()

# 檢查數據
print("Mean Returns:")
print(mean_returns)
print("Standard Deviations:")
print(std_dev)

Mean Returns:
Ticker
AAPL     0.001024
ADBE     0.000354
AMZN     0.000141
GOOGL    0.000573
INTC    -0.000632
META    -0.000248
MSFT     0.000808
NVDA     0.001806
ORCL     0.000839
TSLA     0.002965
dtype: float64
Standard Deviations:
Ticker
AAPL     0.023266
ADBE     0.026219
AMZN     0.024621
GOOGL    0.021748
INTC     0.026506
META     0.030692
MSFT     0.021895
NVDA     0.035262
ORCL     0.021316
TSLA     0.045474
dtype: float64


In [4]:
POPULATION_SIZE = 50
MAX_GENERATIONS = 100
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.2
NUM_STOCKS = len(tickers)

def fitness(chromosome):
    total_return = np.sum(chromosome * mean_returns)
    total_risk = np.sum(chromosome * std_dev)
    num_selected = np.sum(chromosome)

    if num_selected > 5:
        return -1  # 超過 5 支股票，適應度設為負數

    fitness_value = total_return - 0.5 * total_risk + 0.05 * num_selected
    return max(fitness_value, 0)

def initialize_population():
    population = []
    for _ in range(POPULATION_SIZE):
        chromosome = np.zeros(NUM_STOCKS)
        num_selected = random.randint(2, 5)
        selected_indices = random.sample(range(NUM_STOCKS), num_selected)
        chromosome[selected_indices] = 1
        population.append(chromosome)
    return np.array(population)

def select(population, fitness_scores):
    total_fitness = np.sum(fitness_scores)
    probabilities = fitness_scores / total_fitness if total_fitness > 0 else np.ones(len(fitness_scores)) / len(fitness_scores)
    probabilities = np.nan_to_num(probabilities, nan=0.0)
    selected_index = np.random.choice(range(POPULATION_SIZE), size=POPULATION_SIZE, p=probabilities)
    return population[selected_index]

def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, NUM_STOCKS - 1)
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
        return child1, child2
    return parent1, parent2

def mutate(chromosome):
    for i in range(NUM_STOCKS):
        if random.random() < MUTATION_RATE:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

def genetic_algorithm():
    population = initialize_population()
    best_solution = None
    best_fitness = float('-inf')

    for generation in range(MAX_GENERATIONS):
        fitness_scores = np.array([fitness(chromosome) for chromosome in population])
        if fitness_scores.max() > best_fitness:
            best_fitness = fitness_scores.max()
            best_solution = population[fitness_scores.argmax()]
        population = select(population, fitness_scores)
        new_population = []
        for i in range(0, POPULATION_SIZE, 2):
            parent1, parent2 = population[i], population[i+1]
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1), mutate(child2)])
        population = np.array(new_population)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness:.4f}")

    return best_solution, best_fitness

best_solution, best_fitness = genetic_algorithm()


Generation 1: Best Fitness = 0.1891
Generation 2: Best Fitness = 0.1970
Generation 3: Best Fitness = 0.1970
Generation 4: Best Fitness = 0.1970
Generation 5: Best Fitness = 0.1970
Generation 6: Best Fitness = 0.1970
Generation 7: Best Fitness = 0.1970
Generation 8: Best Fitness = 0.1970
Generation 9: Best Fitness = 0.1970
Generation 10: Best Fitness = 0.1970
Generation 11: Best Fitness = 0.1970
Generation 12: Best Fitness = 0.1970
Generation 13: Best Fitness = 0.1970
Generation 14: Best Fitness = 0.1970
Generation 15: Best Fitness = 0.1970
Generation 16: Best Fitness = 0.1970
Generation 17: Best Fitness = 0.1970
Generation 18: Best Fitness = 0.1970
Generation 19: Best Fitness = 0.1970
Generation 20: Best Fitness = 0.1970
Generation 21: Best Fitness = 0.1970
Generation 22: Best Fitness = 0.1970
Generation 23: Best Fitness = 0.1970
Generation 24: Best Fitness = 0.1970
Generation 25: Best Fitness = 0.1970
Generation 26: Best Fitness = 0.1970
Generation 27: Best Fitness = 0.1970
Generation

In [5]:
# 計算並打印結果
selected_stocks = [tickers[i] for i in range(NUM_STOCKS) if best_solution[i] == 1]
daily_return = np.sum(best_solution * mean_returns)
daily_risk = np.sum(best_solution * std_dev)

# 年化回報率與風險
annual_return = daily_return * 252  # 年化回報率
annual_risk = daily_risk * np.sqrt(252)  # 年化風險

# 打印結果
print("\n最佳投資組合:")
print(f"選股結果: {selected_stocks}")
print(f"每日回報率: {daily_return:.4f}")
print(f"每日風險: {daily_risk:.4f}")
print(f"年化回報率: {annual_return:.4f}")
print(f"年化風險: {annual_risk:.4f}")
print(f"適應度: {best_fitness:.4f}")



最佳投資組合:
選股結果: ['AAPL', 'GOOGL', 'AMZN', 'META', 'ORCL']
每日回報率: 0.0034
每日風險: 0.1128
年化回報率: 0.8532
年化風險: 1.7914
適應度: 0.1970
